In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet50
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
# Initialize Inception model
import torchvision.models as models
import numpy as np

In [ ]:
pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 66.6 MB/s eta 0:00:00


In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Data preprocessing
# Resize images to 299x299
transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
# Load CIFAR-10 dataset
train_dataset = CIFAR10(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True, num_workers=4)

100%|██████████| 170498071/170498071 [00:03<00:00, 48968265.03it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
# Initialize ResNet50 model
import torchvision.models as models
model = models.inception_v3(pretrained=True)
num_classes = 10
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

for param in model.parameters():
    param.requires_grad = False


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:01<00:00, 56.0MB/s]


In [ ]:
import torch
import pennylane as qml

dev = qml.device("default.qubit", wires=4)

@qml.qnode(dev)
def quantum_net(q_input_features, q_weights_flat):
    # Reshape weights
    q_weights = q_weights_flat.reshape((4, 6))  # Assuming 6 parameters per qubit

    # Quantum circuit
    for i in range(4):
        qml.Rot(q_input_features[i][0], q_input_features[i][1], q_input_features[i][2], wires=i)
        qml.RX(q_weights[i][0], wires=i)
        qml.RY(q_weights[i][1], wires=i)
        qml.RZ(q_weights[i][2], wires=i)
        qml.RX(q_weights[i][3], wires=i)  # Additional gate
        qml.RY(q_weights[i][4], wires=i)  # Additional gate
        qml.RZ(q_weights[i][5], wires=i)  # Additional gate

    return [qml.expval(qml.PauliZ(i)) for i in range(4)]



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
"""

class QuantumNetLayer(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


# Assuming the input_size and output_size are known
quantum_layer = QuantumNetLayer(input_size=2048, output_size=10)  # Assuming output_size for CIFAR-10 is 10
"""
# Inside the QuantumNetLayer class
class QuantumNetLayer(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.q_weights_flat = nn.Parameter(torch.randn(4, 6).to(device))  # Define trainable quantum weights
        self.fc = nn.Linear(4, output_size).to(device)  # Adjust the input size to match the output of the quantum circuit

    def forward(self, x):
        q_features = quantum_net(torch.flatten(x, start_dim=1), self.q_weights_flat)
        q_features_tensor = torch.tensor(q_features, dtype=torch.float32).unsqueeze(0).to(device)
        x = self.fc(q_features_tensor)
        return x

# Assuming the output_size is known
quantum_layer = QuantumNetLayer(input_size=2048, output_size=10)  # Adjust the input size to match the output of the quantum circuit

# Replace the fully connected layer in the ResNet model with the QuantumNetLayer
model.fc = quantum_layer



In [ ]:
# Set the device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [ ]:
print(model)

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [ ]:
# Loss function and optimizer
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)
criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(model.parameters(), lr=0.001)
#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)


In [ ]:
"""
# Training loop
num_epochs = 5
total_step = len(train_loader)
for epoch in range(num_epochs):
    # Inside the training loop
    # Inside the training loop
    # Inside the training loop
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        # Forward pass
        inception_outputs, _ = model(images)

        # Ensure outputs and labels have the same batch size
        labels = labels[:inception_outputs.shape[0]]  # Trim labels if needed to match batch size
        loss = criterion(inception_outputs, labels)

        # Backward and optimize
        loss.backward()
        optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item()}')


        # Adjust learning rate
    # scheduler.step()

"""
# Training loop
num_epochs = 5
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        # Forward pass
        inception_outputs, _ = model(images)
        labels = labels[:inception_outputs.shape[0]]
        loss = criterion(inception_outputs, labels)

        # Add L2 regularization term to the loss
        l2_lambda = 0.001
        l2_reg = sum(l2_lambda * p.norm(2) for p in model.parameters())
        loss += l2_reg

        # Backward and optimize
        loss.backward()
        optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item()}')

Epoch [1/5], Step [1/782], Loss: 5.539053916931152
Epoch [1/5], Step [2/782], Loss: 5.2971649169921875
Epoch [1/5], Step [3/782], Loss: 5.078428268432617
Epoch [1/5], Step [4/782], Loss: 4.835585117340088
Epoch [1/5], Step [5/782], Loss: 4.63450813293457
Epoch [1/5], Step [6/782], Loss: 4.398934364318848
Epoch [1/5], Step [7/782], Loss: 4.4112114906311035
Epoch [1/5], Step [8/782], Loss: 4.243680953979492
Epoch [1/5], Step [9/782], Loss: 4.286917209625244
Epoch [1/5], Step [10/782], Loss: 4.920343399047852
Epoch [1/5], Step [11/782], Loss: 4.409223556518555
Epoch [1/5], Step [12/782], Loss: 4.631003379821777
Epoch [1/5], Step [13/782], Loss: 4.644822120666504
Epoch [1/5], Step [14/782], Loss: 4.722283363342285
Epoch [1/5], Step [15/782], Loss: 4.825125694274902
Epoch [1/5], Step [16/782], Loss: 4.596438884735107
Epoch [1/5], Step [17/782], Loss: 4.445266246795654
Epoch [1/5], Step [18/782], Loss: 4.789846420288086
Epoch [1/5], Step [19/782], Loss: 4.535282135009766
Epoch [1/5], Step [2

In [ ]:
model.eval()

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Accuracy on the test set: {100 * accuracy:.2f}%')


Accuracy on the test set: 9.27%
